In [1]:
import os
import numpy as np
from scripts.datagen.create_dataset import create_dataset
from scripts.utils.params import *
from scripts.utils.utils import *
from scripts.particle.rungekutta import * 

In [2]:

s_params = SolverParams(final_time=500, time_step=0.5, u0=[0.0, 0.0], theta=0.5, tol=1e-2)
fn_params = FNParams(
    solver_params=s_params, 
    k=8, 
    alpha=0.15, 
    epsilon=1e-2, 
    I=0.125, 
    gamma=0.1, 
    grid_size=(20, 20), 
    solver_name="thetamethod"
)

In [3]:
create_dataset(
    dataset_name="test1", 
    num_samples=8, 
    num_processes=4, 
    fn_params=fn_params, 
    batch_size=6, 
    generate=True, 
    remove_samples=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/samples/sample_1.npy'

In [4]:
# a = np.load('data/dataset/test1/test1_0.npz')['data']
# print(a.shape)
a = np.load('data/dataset/test1/test1_1.npz')['data']
a.shape

(2, 1001, 20, 20, 2)

In [3]:
8%6

2